# Network Creation

In [1]:
%load_ext autoreload
%autoreload

import networkx as nx
import networkx.algorithms as algos
from networkx.algorithms import approximation
from networkTrips import organizeTrips
from networkAlgos import networkAlgos
from timeUtils import clock, elapsed, getDateTime
from collections import Counter
from haversine import haversine
from ioUtils import loadJoblib, saveFile, getFile
from fsUtils import mkDir, mkSubDir, setFile
from pandasUtils import getRowData, getColData, dropColumns, fixType, isDataFrame
from numpyUtils import isNumericDtype
from networkAlgos import networkAlgos
from edgeInfo import edgeInfo
from vertexInfo import vertexInfo
from networkCategories import categories
from place import getPlaceData
from cbsa import getCBSAData
from csa import getCSAData
from metdiv import getMetDivData
from county import getCountyData
from state import getStateData
from geocluster import geoClusters
from geoUtils import convertMetersToLat, convertLatToMeters, convertMetersToLong, convertLongToMeters
from geoclusterUtils import genCenters, genCluster, genClusters, genTripsBetweenClusters

import pandas as pd
pd.set_option("display.max_rows",1000)
pd.set_option('precision', 3)

import warnings
warnings.filterwarnings('ignore')

_, _ = clock("Last Run")

Current Time is Tue Dec 04, 2018 09:47:56 for Last Run


In [2]:
savedir = "/Users/tgadf/Downloads/network"
mkDir(savedir)

'/Users/tgadf/Downloads/network'

# Load/Generate Data

In [3]:
#######################################################################################
# Generate Clusted Data
#######################################################################################
genData = False
if genData:
    cls     = 20
    total   = 500
    genMax  = 75
    distMax = 500
    raw  = genClusters(cls, 250, latRange=[29.8, 30.2], lngRange=[49.8, 50.2], dist="gauss", maxrad=genMax)
    gc   = geoClusters(key="dummy", points=raw, distMax=distMax, debug=False)
    gc.findClusters(seedMin=2, debug=False)
    df   = genTripsBetweenClusters(n=total, gc=gc, returnDF=True)
    df["device"] = "dummy"    
    
    tmpdf = loadJoblib("/Users/tgadfort/Downloads/r4hIDs.p").sample(n=total, replace=True)
    tojoin = tmpdf.sample(cls)
    tojoin["cl"] = ["cl{0}".format(x) for x in range(cls)]

    df['cl'] = df['cl0']
    drops = [x for x in tojoin.columns if x.startswith("Geo1")]
    tojoinCL0 = dropColumns(tojoin, columns=drops, inplace=False)
    test = df.merge(tojoinCL0, on='cl')

    test['cl'] = test['cl1']
    drops = [x for x in tojoin.columns if x.startswith("Geo0")]
    tojoinCL1 = dropColumns(tojoin, columns=drops, inplace=False)
    test = test.merge(tojoinCL1, on='cl')

    gpsdata = test
    dropColumns(gpsdata, columns=["cl", "cl0", "cl1"])
    gpsdata.replace('nan', 0, inplace=True)
else:
    fname = "/Users/tgadf/Downloads/gpsTripsOakRidge.p"
    print("Loading {0}".format(fname))
    gpsdata = loadJoblib(fname)    

_, _ = clock("Last Run")

Loading /Users/tgadf/Downloads/gpsTripsOakRidge.p
Current Time is Tue Dec 04, 2018 09:48:03 for Last Run


## Show Data (if needed)

## Subselect (if needed)

In [4]:
device  = '352252060173789'
gpsdata = gpsdata[gpsdata['device'] == device]
print("Keeping {0} rows".format(gpsdata.shape[0]))

Keeping 3066 rows


# Cluster and Sort Trips

In [5]:
i  = 0
nd = gpsdata['device'].nunique() 
for device, df in gpsdata.groupby('device'):
    print('Key = {0}'.format(device),'\tRun = {0}/{1}'.format(i,nd),'\tTrips = {0}'.format(df.shape[0]))
    i += 1

    #######################################################################################
    # Cluster Geo Data (Lat, Long)
    #######################################################################################
    points         = df[["lat0", "long0"]]
    points.columns = ["lat", "long"]
    pnts           = df[["lat1", "long1"]]
    pnts.columns   = ["lat", "long"]    
    points         = points.append(pnts)



    #######################################################################################
    # Create Clusters
    #######################################################################################
    debug=True
    gc   = geoClusters(key="dummy", points=points, distMax=300, debug=debug)
    gc.findClusters(seedMin=4, debug=debug)
    if debug:
        print("Found {0} clusters using {1} cells and {2} counts".format(gc.getNClusters(), gc.getNCells(), gc.getNCounts()))



    #######################################################################################
    # Set Nearest Clusters
    #######################################################################################
    if debug:
        start, cmt = clock("Finding Nearest Clusters for Start of Trips")
    geoResults = df[['lat0', 'long0']].apply(gc.getNearestClusters, axis=1).values
    df["geo0"] = [x[0] for x in geoResults]
    if debug:
        elapsed(start, cmt)
        start, cmt = clock("Finding Nearest Clusters for End of Trips")
    geoResults = df[['lat1', 'long1']].apply(gc.getNearestClusters, axis=1).values
    df["geo1"] = [x[0] for x in geoResults]    
    if debug:
        elapsed(start, cmt)


    #######################################################################################
    # Organize Trips for Network
    #######################################################################################
    trips = organizeTrips(df=df, gc=gc, debug=True, requireGood=False)

Key = 352252060173789 	Run = 0/1 	Trips = 3066
Current Time is Tue Dec 04, 2018 09:48:03 for Converting 6132 Points To Correct Format
Data has correct format with a (6132, 2) shape.
Current Time is Tue Dec 04, 2018 09:48:03 for Done with Converting 6132 Points To Correct Format
Process [Done with Converting 6132 Points To Correct Format] took 0 seconds.
Current Time is Tue Dec 04, 2018 09:48:03 for Finding Geohash (BitLen=8) Values from 6132 Points
Current Time is Tue Dec 04, 2018 09:48:03 for Done with Finding Geohash (BitLen=8) Values from 6132 Points
Process [Done with Finding Geohash (BitLen=8) Values from 6132 Points] took 0 seconds.
Current Time is Tue Dec 04, 2018 09:48:03 for Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Current Time is Tue Dec 04, 2018 09:48:03 for Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame
Process [Done with Finding Geohash (BitLen=8) Frequency Values from Geohash DataFrame] took 0 seconds.
Current Time i

# Saved Data

In [6]:
# Save trips/gc if needed
deviceDir = mkSubDir(savedir, device)
tripsfile = setFile(deviceDir, "trips.p")
gcfile    = setFile(deviceDir, "gc.p")
loadTrips=False
if loadTrips:
    trips = getFile(tripsfile)
    gc    = getFile(gcfile)
else:
    print("Saving to {0}".format(deviceDir))
    saveFile(ifile=gcfile, idata=gc)
    saveFile(ifile=tripsfile, idata=trips)
    
_, _ = clock("Last Run")

Saving to /Users/tgadf/Downloads/network/352252060173789
  --> This file is 332.2kB.
  --> This file is 247.4kB.
Current Time is Tue Dec 04, 2018 09:48:24 for Last Run


In [7]:
# Show data if needed
df.head()

,device,Start,End,total_miles,heading0,lat0,long0,heading1,lat1,long1,...,Geo1CENSUSPlace,Geo0CENSUSMetdiv,Geo1CENSUSMetdiv,Geo0CENSUSCsa,Geo1CENSUSCsa,Geo0CENSUSCbsa,Geo1CENSUSCbsa,geo0,geo1,Date
946,352252060173789,2018-02-01 15:56:11,2018-02-01 17:08:03,1.5,60,35.894,-84.173,66,35.901,-84.151,...,4740000,0,0,314,314,28940,28940,cl33,cl37,2018-02-01
953,352252060173789,2018-03-17 20:06:12,2018-03-17 20:08:00,0.1,66,35.901,-84.149,72,35.901,-84.151,...,4740000,0,0,314,314,28940,28940,cl37,cl37,2018-03-17
4374,352252060173789,2017-11-01 11:26:21,2017-11-01 11:32:45,1.8,348,35.720,-84.342,138,35.718,-84.367,...,4743780,0,0,314,314,28940,28940,cl23,cl7,2017-11-01
4376,352252060173789,2017-09-08 15:12:41,2017-09-08 15:43:06,10.3,48,35.783,-84.279,252,35.718,-84.368,...,4743780,0,0,314,314,28940,28940,cl72,cl7,2017-09-08
4378,352252060173789,2018-05-09 13:22:14,2018-05-09 13:46:36,4.3,288,35.725,-84.343,72,35.718,-84.368,...,4743780,0,0,314,314,28940,28940,cl0,cl7,2018-05-09


# Driver Network

In [164]:
class network():
    def __init__(self, directed=True, debug=False):
        self.debug = debug
        self.directed = directed
        
        self.orderedEdges    = None
        self.edgeDict        = None
        self.orderedVertices = None
        self.nodeDict        = None
        
        if self.directed is True:
            self.g = nx.DiGraph()
        else:
            self.g = nx.Graph()

        self.eInfo            = edgeInfo(self.g, self.debug)
        self.getEdges         = self.eInfo.getEdges
        self.getEdge          = self.eInfo.getEdgeData
        self.getEdgeByName    = self.eInfo.getEdgeDataByName
        self.getEdgeAttrsGroups     = self.eInfo.getAttrGroups
        self.setEdgeFeature   = self.eInfo.setEdgeFeature        
        self.getEdgeFeature   = self.eInfo.getEdgeFeature
        self.getEdgeFeatures  = self.eInfo.getEdgeFeatures
        self.getEdgeNum       = self.eInfo.getEdgeNumByName
        
        self.vInfo                  = vertexInfo(self.g, self.debug)
        self.getVertices            = self.vInfo.getVertices
        self.getVertex              = self.vInfo.getVertexData
        self.getVertexByName        = self.vInfo.getVertexDataByName
        self.getVertexAttrsGroups   = self.vInfo.getAttrGroups
        self.setVertexFeature       = self.vInfo.setVertexFeature
        self.getVertexFeature       = self.vInfo.getVertexFeature
        self.getVertexFeatures      = self.vInfo.getVertexFeatures
        self.getVertexNum           = self.vInfo.getVertexNumByName
            
    def setDebug(self, debug):
        self.debug = debug
        
    def getNetwork(self):
        return self.g
    
    
    def update(self, debug=False):
        if debug:
            print("Updating Vertices/Edges")
        self.eInfo.orderEdges(debug=debug)
        self.vInfo.orderVertices(debug=debug)
        
            
    def flattenAttrs(self, debug=False):
        if debug:
            print("Flattening Vertices/Edges")
        self.eInfo.flattenEdgeAttrs(debug=debug)
        self.vInfo.flattenVertexAttrs(debug=debug)
        
    
    def collectAttrs(self, debug=False):
        if debug:
            print("Collecting Vertices/Edges")
        self.eInfo.collectEdgeAttrs(debug=debug)
        self.vInfo.collectVertexAttrs(debug=debug)
    
        if debug:
            print("Creating Vertex Attrs DataFrame")
        self.vInfo.createVertexAttrsDataFrame(debug=debug)
    
        if debug:
            print("Creating Edge Attrs DataFrame")
        self.eInfo.createEdgeAttrsDataFrame(debug=debug)
    
    
    
    
    ################################################################################################
    # Show Network Data
    ################################################################################################    
    def showVertices(self):
        for nodename,node in self.g.nodes_iter(data=True):
            print(nodename,'\t',node)
                
    def showEdges(self):
        for edgename,edge in self.g.adj.items():
            print(edgename,'\t',edge)
                
                

        
    ################################################################################################
    # Vertices / Nodes / Location (Initial Functions)
    ################################################################################################    
    def addVertex(self, name, attrs={}):
        self.g.add_node(u=name, attr_dict=attrs)
        if self.debug:
            print("  Added node: [{0}]".format(", ".join(names)))
                    
    def updateVertexAttrs(self, attrs, debug=False):
        if debug:
            print("Updating Vertex Attributes")
        if not isinstance(attrs, dict):
            print("Cannot add vertex attrs because the input is not a dict")
            return
        nx.set_node_attributes(G=self.g, values=attrs, name=None)
            
        
        
    ################################################################################################
    # Edges / Trips (Initial Functions)
    ################################################################################################    
    def addEdge(self, names, attrs={}, sort=False):
        if not isinstance(names, (tuple,list,set)):
            print("Cannot add edge {0} because the names need to come in a tuple/list/set.".format(names))
            return
        if len(names) == 2:
            if sort is True:
                names = sorted([str(x) for x in names])
            else:
                names = [str(x) for x in names]
        else:
            print("Cannot add edge {0} because we need two entries in the tuple/list/set.".format(names))
            return
        
        self.g.add_edge(names[0], names[1], attr_dict=attrs)
        if self.debug:
            print("  Added edge: [{0}]".format(", ".join(names)))
            
    def updateEdgeAttrs(self, attrs):
        if not isinstance(attrs, dict):
            print("Cannot add edge attrs because the input is not a dict")
            return
        nx.set_edge_attributes(G=self.g, values=attrs)

In [199]:
class driverNetwork(network):
    def __init__(self, trips):
        network.__init__(self, directed=False, debug=False)
        
        self.categories        = categories(debug)
        self.getCategories     = self.categories.getCategories
        self.getCategory       = self.categories.getCategory
        self.getPermCategories = self.categories.getPermCategories
        self.getPermCategory   = self.categories.getPermCategory  
        self.getHomeRatioCategory = self.categories.getHomeRatioCategory
        self.getIntervalCategory = self.categories.getIntervalCategory
        
        self.nodeAttrs = None
        self.edgeAttrs = None
        self.netAttrs  = None
                
        if trips is not None:
            if isinstance(trips, dict):
                self.name          = trips.get('device')
                self.edgeMetrics   = trips.get('edgeMetrics')
                self.vertexMetrics = trips.get('vertexMetrics')
                self.vertexMetrics = {str(k): v for k,v in self.vertexMetrics.items()}
                self.homeMetrics   = trips.get('homeMetrics')
                print("Creating a driver network with {0} vertices and {1} edges.".format(len(self.vertexMetrics), len(self.edgeMetrics)))
            else:
                raise ValueError("Input trips must be a dictionary of edgeMetrics, vertexMetrics, and homeMetrics (optional)")
        else:
            raise ValueError("Input trips is None!")

            
    ################################################################################################
    # Getters
    ################################################################################################    
    def getNodeAttrs(self):
        return self.nodeAttrs
        
    def getNodeDict(self):
        return self.vInfo.nodeDict
        
    def getEdgeAttrs(self):
        return self.edgeAttrs
        
    def getNetAttrs(self):
        return self.netAttrs
        
            
    ####################################################################################
    # Create Network
    ####################################################################################
    def create(self, debug=False):
        if debug:
            print("Creating Network Attributes")
        for edgename,edgedata in self.edgeMetrics.items():
            self.addEdge(edgename, edgedata)
        self.updateVertexAttrs(self.vertexMetrics, debug=debug)
        self.update(debug=debug)
        self.flattenAttrs(debug=debug)
        self.collectAttrs(debug=debug)

        
    ####################################################################################
    # Compute Network Attributes
    ####################################################################################
    def computeNetworkAttrs(self, level=2, debug=False):
        if debug:
            if level == 1:
                print("Computing Network Attributes (simple)")
            elif level == 2:
                print("Computing Network Attributes (medium)")
            elif level == 3:
                print("Computing Network Attributes (hard)")
        self.netAlgos = networkAlgos()
        results = self.netAlgos.compute(self.g, level=level, debug=debug)
        self.nodeAttrs = results['Nodes']
        self.edgeAttrs = results['Edges']
        self.edgeAttrs['edge_weight'] = self.eInfo.getEdgeWeights().values() # add weights
        self.netAttrs  = results['Net']
        if debug:
            print("  Created {0} attributes for {1} vertices".format(self.nodeAttrs.shape[1], self.nodeAttrs.shape[0]))
            print("  Created {0} attributes for {1} edges".format(self.edgeAttrs.shape[1], self.edgeAttrs.shape[0]))
            print("  Created {0} attributes for the network".format(len(self.netAttrs)))


    ####################################################################################
    # Perform Lookup for Census Data
    ####################################################################################
    def fillVertexCensusData(self, debug=False):
        if debug:
            print("Filling Vertex Census Data")
        verydebug=False
        
        censusKeys = [k for k,v in self.getVertexAttrsGroups().items() if v == "Census"]
        getCensusData = {"CENSUSCbsa": getCBSAData, "CENSUSCsa": getCSAData, "CENSUSCounty": getCountyData, 
                         "CENSUSMetdiv": getMetDivData, "CENSUSPlace": getPlaceData, "CENSUSState": getStateData}
        for key in censusKeys:
            if getCensusData.get(key) is None:
                continue
            for vertexName in self.getVertices():
                vertex = self.getVertexByName(vertexName, 'attr')
                if verydebug:
                    print("  --> Vertex Name {0}".format(vertexName))

                value   = vertex[key]
                
                
                if isinstance(value, list):
                    try:
                        #mc    = value.most_common(1)
                        value = value[0][0]
                    except:
                        print("There was an error getting most common {0}".format(key))
                        value = None        
                else:
                    print("Input {0} is type {1}".format(value, type(value)))
                    
                try:
                    lookup       = getCensusData[key](str(value))
                    features     = self.categories.getFeatures(key.replace("CENSUS", "Census"), lookup, debug)
                except:
                    raise ValueError("Something went wrong with census lookup for {0} and value {1}".format(key, value))

                for lookupName,lookupValue in features.items():
                    featureName = "".join([key,lookupName])
                    self.setVertexFeature(vertexName=vertexName, category="Census", key=featureName, value=lookupValue)
                
                if verydebug is True:
                    print("\t: {0}, {1} == {2} ({3})".format(key, value, lookup, features))
                    
                    
        ### Fill CENSUS Region
        for vertexName in self.getVertices():
            state  = self.getVertexFeature(vertexName=vertexName, category="Census", key="State")
            region = self.categories.getFeatures("CensusRegion", state, debug=True)
            value  = region['Region']
            self.setVertexFeature(vertexName=vertexName, category="Census", key="Region", value=value)
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")
            

            
    ####################################################################################
    # Format and Fill GeoSpatial Data
    ####################################################################################
    def fillVertexGeospatialData(self, debug=False):
        if debug:
            print("Filling Vertex Geospatial Data")
        verydebug=False
        
        groupings = ["HEREPOI", "OSM", "Roads", "Rail", "Terminal"]
        for grouping in groupings:
            keys = [k for k,v in self.getVertexAttrsGroups().items() if v == grouping]            
            for vertexName in self.getVertices():
                vertex = self.getVertexByName(vertexName, 'attr')
                if verydebug:
                    print("  --> Vertex Number {0} and ID {1}".format(vertexNum, vertexID))

                for key in keys:
                    value   = vertex[key]

                    result = None
                    if isinstance(value, list):
                        try:
                            test = value[0][0]
                            if test is None:
                                result = 0 #'N'
                            else:
                                if test == 1.0:
                                    result = 1 #'Y'
                                else:
                                    result = 0 #'N'
                        except:
                            result = 0 #'N'
                    else:
                        print("Input {0} is type {1}".format(value, type(value)))

                    self.setVertexFeature(vertexName=vertexName, category="GeoSpatial", key=key, value=result)
                    if verydebug is True:
                        print("\t: {0}, {1} == {2}".format(key, value, result))
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")            
            

            
    ####################################################################################
    # Format and Fill Internal Vertex Data
    ####################################################################################
    def fillVertexInternalData(self, debug=False):
        if debug:
            print("Filling Vertex Internal Data")
        verydebug=False

        keys = [k for k,v in self.getVertexAttrsGroups().items() if v == "General"]
        for vertexName in self.getVertices():
            vertex = self.getVertexByName(vertexName, 'attr')
            if verydebug:
                print("  --> Vertex Number {0}".format(vertexName))

            for key in keys:
                value   = vertex[key]
                feature = self.categories.getFeatures(key, value, debug)
                if isinstance(feature, dict):
                    if feature.get('Name') is None:
                        feature = value
                    else:
                        feature = feature['Name']
                self.setVertexFeature(vertexName=vertexName, category="Internal", key=key, value=feature)
                   
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")                  
            

            
    ####################################################################################
    # Format and Fill Network Algos Vertex Data
    ####################################################################################
    def fillVertexNetworkData(self, debug=False):
        if debug:
            print("Filling Vertex Network Data")
        verydebug=False

        vertexNetworkDF = self.getVertexNetworkDataFrame()
        if not isDataFrame(vertexNetworkDF):
            if debug:
                print("There is no vertex network DataFrame!")
            return
        
        for vertexName in self.getVertices():
            vertexData = getRowData(vertexNetworkDF, rownames=vertexName)
            if verydebug:
                print("  --> Vertex Name {0}".format(vertexName))

            for key in vertexData.index:
                value = vertexData[key]
                featureName = ''.join([s.title() for s in key.split("_")])
                self.setVertexFeature(vertexName=vertexName, category="Network", key=featureName, value=value)
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")                      
            

            
            
            
    ####################################################################################
    ####################################################################################
    # Format and Fill Edge Vertex Data
    ####################################################################################                    
    ####################################################################################
    def fillEdgeVertexData(self, debug=False):
        if debug:
            print("Filling Edge Data")
        verydebug=False

        for edgeName in self.getEdges():
            try:
                features = [self.getVertexByName(x, 'feat') for x in tuple(edgeName)]
            except:
                if debug:
                    print("  There are no vertex features!")
                return
            categories = features[0].keys()
            categories = ["Census", "GeoSpatial"]
            for category in categories:
                featureNames = features[0][category]
                for featureName in featureNames:
                    values = [features[i][category][featureName] for i in range(2)]
                    self.setEdgeFeature(edgeName=edgeName, category=category, key=featureName, value=values)
                    if verydebug is True:
                        print("\t: {0}, {1}, {2}".format(edgeName, key, values))
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")                              
            

    def fillEdgeNetworkData(self, debug=False):
        if debug:
            print("Filling Edge Network Data")
        verydebug=False

        edgeNetworkDF = self.getEdgeNetworkDataFrame()
        if not isDataFrame(edgeNetworkDF):
            if debug:
                print("There is no edge network DataFrame!")
            return
        
        for edgeName in self.getEdges():
            #edgeData = getRowData(edgeNetworkDF, rownames=str(tuple(edgeName)))
            edgeData = getRowData(edgeNetworkDF, rownums=list(edgeNetworkDF.index).index(edgeName)) # Still ???
            if verydebug:
                print("  --> Edge Name {0}".format(edgeName))

            for key in edgeData.index:
                value = edgeData[key]
                featureName = ''.join([s.title() for s in key.split("_")])
                self.setEdgeFeature(edgeName=edgeName, category="Network", key=featureName, value=value)
                    
        if verydebug:
            raise ValueError("Stoppping after verydebug is True")                            
            

    def fillEdgeInternalData(self, debug=False):
        if debug:
            print("Filling Edge Internal Data")
        verydebug=False

        for edgeName in self.getEdges():
            edgeDataDF = self.getEdgeByName(edgeName, 'attr')
            if verydebug:
                print("  --> Edge Name {0}".format(edgeName))
                
            if not isDataFrame(edgeDataDF):
                if debug:
                    print("There is no edge internal DataFrame!")
                return
            
            keys = edgeDataDF.columns
            for key in keys:
                value   = getColData(edgeDataDF, colnames=key)[0]
                feature = self.categories.getFeatures(key, value, debug)
                if isinstance(feature, dict):
                    if feature.get('Name') is None:
                        continue
                    else:
                        feature = feature['Name']
                if isinstance(feature, list):
                    continue
                self.setEdgeFeature(edgeName=edgeName, category="Internal", key=key, value=feature)
            
            
       
        
    #################################################################################################################
    # DataFrame Functions
    #################################################################################################################
    def getVertexInternalDataFrame(self, debug=False):
        return self.vInfo.vertexAttrsDF

    def getVertexExternalDataFrame(self, debug=False):
        return self.vInfo.vertexAttrsDF

    def getVertexNetworkDataFrame(self, debug=False):
        return self.nodeAttrs

    def getEdgeInternalDataFrame(self, debug=False):
        return self.eInfo.edgeAttrsDF

    def getEdgeNetworkDataFrame(self, debug=False):
        return self.edgeAttrs

    def getNetworkDataFrame(self, debug=False):
        return self.netAttrs

In [201]:
%load_ext autoreload
%autoreload

from edgeInfo import edgeInfo
from vertexInfo import vertexInfo
from networkCategories import categories
from networkAlgos import networkAlgos

dn = driverNetwork(trips)
dn.create(debug=True)
dn.computeNetworkAttrs(debug=True, level=1)
dn.fillVertexCensusData(debug=True)
dn.fillVertexGeospatialData(debug=True)
dn.fillVertexInternalData(debug=True)
dn.fillVertexNetworkData(debug=True)
dn.fillEdgeInternalData(debug=True)
dn.fillEdgeVertexData(debug=True)
dn.fillEdgeNetworkData(debug=True)
g = dn.getNetwork()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Creating a driver network with 88 vertices and 575 edges.
Creating Network Attributes
Updating Vertex Attributes
Updating Vertices/Edges
Ordering Edges by Weight
Ordering Vertices by Centrality
Flattening Vertices/Edges
Flattening Vertex Attributes
Creating Vertex Attrs DataFrame
Cleaning Vertex Attribute Names
Creating Edge Attrs DataFrame
Cleaning Edge Attribute Names
Computing Network Attributes (simple)
Computing Network Algorithms
Running network algorithms
Creating Algorithm Results DataFrame for Vertices
Creating Algorithm Results DataFrame for Edges
  Created 30 attributes for 88 vertices
  Created 2 attributes for 575 edges
  Created 46 attributes for the network
Filling Vertex Census Data
Filling Vertex Geospatial Data
Filling Vertex Internal Data
Filling Vertex Network Data
Filling Edge Internal Data
Filling Edge Data
Filling Edge Network Data


In [202]:
dn.getVertexByName('cl33', 'feat')
#dn.getEdgeFeatures(('cl33', 'cl13'))
#edgeNetworkDF = dn.getEdgeNetworkDataFrame()
#dn.getVertexByName('cl33', 'raw')
#dn.getNodeDict()

{'Census': {'Cbsa': 'Knoxville, TN',
  'CbsaType': 'Metro',
  'CbsaPop': 'Big',
  'CbsaHousing': 'Big',
  'CbsaArea': 'Mid',
  'Csa': 'Knoxville-Sevierville-La Follette, TN',
  'CsaType': 'CSA',
  'CsaPop': 'Mid',
  'CsaHousing': 'Mid',
  'CsaArea': 'Mid',
  'Metdiv': None,
  'MetdivType': 'Other',
  'MetdivPop': 'RealSmall',
  'MetdivHousing': 'RealSmall',
  'MetdivArea': 'RealSmall',
  'Place': 'Farragut',
  'PlaceType': 'Town',
  'PlacePop': 'Big',
  'PlaceHousing': 'Big',
  'PlaceArea': 'Small',
  'County': 'Knox County',
  'CountyType': 'A',
  'CountyPop': 'Big',
  'CountyHousing': 'Big',
  'CountyArea': 'Small',
  'State': 'Tennessee',
  'StateRegion': 'SOUTHEastSouthCentral',
  'Region': 'SOUTHEastSouthCentral'},
 'GeoSpatial': {'Attraction': 0,
  'Auto': 0,
  'Building': 0,
  'College': 0,
  'Commercial': 0,
  'Cycling': 0,
  'Entertainment': 0,
  'Fastfood': 0,
  'Fuel': 0,
  'Grocery': 0,
  'Industrial': 0,
  'Lodging': 0,
  'Medical': 0,
  'Municipal': 0,
  'Parking': 0,
  '

In [168]:
getRowData(edgeNetworkDF, rownames=[('cl0', 'cl1')])

,edge_betweenness_centrality,edge_weight
"(cl0, cl1)",0.017,368


# Network Features

In [203]:
class networkFeatures():
    def __init__(self, dn):
        self.dn = dn
        self.features = {}
        
        self.categories        = categories(debug)
        self.getCategories     = self.categories.getCategories
        self.getCategory       = self.categories.getCategory
        self.getPermCategories = self.categories.getPermCategories
        self.getPermCategory   = self.categories.getPermCategory  
        self.getHomeRatioCategory = self.categories.getHomeRatioCategory
        self.getIntervalCategory = self.categories.getIntervalCategory
        
        

    #################################################################################################################
    #################################################################################################################
    # Vertex Counts
    #################################################################################################################
    #################################################################################################################
    def fillVertexCensusCounts(self, debug=False):
        if debug:
            print("Filling Vertex Census Counts")
            
            
        from collections import Counter
        vertexCounts = None

        for vertexNum,vertexName in enumerate(self.dn.getVertices()):
            vertexData = dn.getVertexByName(vertexName, 'feat').get("Census")
            if vertexData is None:
                raise ValueError("Could not get {0} category from vertex data!".format(category))
                
            if vertexCounts is None:
                featureNames = list(vertexData.keys())
                vertexCounts = {}
                for featureName in featureNames:
                    vertexCounts[featureName] = Counter()
            
            for featureName,value in vertexData.items():
                vertexCounts[featureName][value] += 1
                  
                            
        retval = {}
        nV = len(dn.getVertices())
        for featureName,counts in vertexCounts.items():
            key = featureName.replace("Census", "")
            key = key.replace("CENSUS", "")
            if key.endswith("Name"):
                key = key[:-4]
            
            retval[key] = {}
            retval[key]["N"] = len(counts)
            try:
                mc = counts.most_common(1)[0]
                retval[key]["MostCommon"]         = mc[0]
                retval[key]["MostCommonFraction"] = mc[1]/nV
            except:
                retval[key]["MostCommon"]         = None
                retval[key]["MostCommonFraction"] = None
    

        if debug:
            print("  Filling Vertex Census Counts")
             
        self.features["Vertex_Census_Counts"] = retval        

        
    def fillVertexCategoryCounts(self, category, debug=False):
        if debug:
            print("Filling Vertex {0} Counts".format(category))
            
        featureNames = None
            
        from collections import Counter
        vertexCounts = {"N": {}, 3: {}, 10: {}, 25: {}}

        for vertexNum,vertexName in enumerate(self.dn.getVertices()):
            vertexData = dn.getVertexByName(vertexName, 'feat').get(category)
            if vertexData is None:
                raise ValueError("Could not get {0} category from vertex data!".format(category))
            
            if featureNames is None:
                featureNames = list(vertexData.keys())
                for featureName in featureNames:
                    featCats = self.getCategories(featureName)
                    if featCats is not None:
                        if featCats == ['Y', 'N']:
                            key = featureName
                            for cutoff in ["N",3,10,25]:
                                vertexCounts[cutoff][key]  = 0
                        else:
                            for cat in featCats:
                                key = "".join([featureName,cat])
                                for cutoff in ["N",3,10,25]:
                                    vertexCounts[cutoff][key]  = 0
                        
            
            for featureName in featureNames:                
                value = vertexData[featureName]
                featCats = self.getCategories(featureName)
                if featCats is not None:
                    if featCats == ['Y', 'N']:
                        key = featureName
                        vertexCounts["N"][key] += int(value == 'Y')
                        for cutoff in [3,10,25]:
                            if vertexNum < cutoff:
                                vertexCounts[cutoff][key] += int(value == 'Y')
                    elif value in featCats:
                        key = "".join([featureName,value])
                        vertexCounts["N"][key] += 1
                        for cutoff in [3,10,25]:
                            if vertexNum < cutoff:
                                vertexCounts[cutoff][key] += 1
                    else:
                        raise ValueError("Value [{0}] not in [{1}] for feature [{2}]".format(value, featCats, featureName))
                            
                            
        retval = {}
        for cutoff,cutoffData in vertexCounts.items():
            for key,value in cutoffData.items():
                if retval.get(key) is None:
                    retval[key] = {}
                if isinstance(cutoff, int):
                    retval[key]["".join(["Top", str(cutoff)])] = value
                else:
                    retval[key][cutoff] = value
                    
        if debug:
            print("  Filling Vertex {0} Counts for {1} Cutoff Values".format(category, len(retval)))
                                
        self.features["Vertex_{0}_Counts".format(category)] = retval
        
        
    def fillVertexInternalCounts(self, debug=False):
        self.fillVertexCategoryCounts(category="Internal", debug=debug)
        
    def fillVertexGeoSpatialCounts(self, debug=False):
        self.fillVertexCategoryCounts(category="GeoSpatial", debug=debug)
        
            

        
        

    #################################################################################################################
    #################################################################################################################
    # Edge Counts
    #################################################################################################################
    #################################################################################################################
    def fillEdgeCensusCounts(self, debug=False):
        if debug:
            print("Filling Edge Census Counts")
            
            
        from collections import Counter
        edgeCounts = None

        for edgeNum,edgeName in enumerate(self.dn.getEdges()):
            edgeData = dn.getEdgeByName(edgeName, 'feat').get("Census")
            if edgeData is None:
                raise ValueError("Could not get {0} category from edge data!".format(category))
                
            if edgeCounts is None:
                featureNames = list(edgeData.keys())
                edgeCounts = {}
                for featureName in featureNames:
                    edgeCounts[featureName] = Counter()
            
            for featureName,value in edgeData.items():
                featureValue = " <-> ".join(sorted([str(x) for x in value]))
                edgeCounts[featureName][featureValue] += 1
                  
                            
        retval = {}
        nE = len(dn.getEdges())
        for key,counts in edgeCounts.items():            
            retval[key] = {}
            retval[key]["N"] = len(counts)
            try:
                mc = counts.most_common(1)[0]
                retval[key]["MostCommon"]         = mc[0]
                retval[key]["MostCommonFraction"] = mc[1]/nE
            except:
                retval[key]["MostCommon"]         = None
                retval[key]["MostCommonFraction"] = None
    

        if debug:
            print("  Filling Edge Census Counts")
             
        self.features["Edge_Census_Counts"] = retval               

     

        
    def fillEdgeGeoSpatialCounts(self, debug=False):
        category = "GeoSpatial"
        if debug:
            print("Filling edge {0} Counts".format(category))
            
        featureNames = None
            
        from collections import Counter
        edgeCounts = {"N": {}, 3: {}, 10: {}, 25: {}}

        for edgeNum,edgeName in enumerate(self.dn.getEdges()):
            edgeData = dn.getEdgeByName(edgeName, 'feat').get(category)
            if edgeData is None:
                raise ValueError("Could not get {0} category from edge data!".format(category))
            
            if featureNames is None:
                featureNames = list(edgeData.keys())
                for featureName in featureNames:
                    key = featureName
                    for cutoff in ["N",3,10,25]:
                        edgeCounts[cutoff][key]  = 0
                        
            
            for featureName in featureNames:
                key = featureName
                value = edgeData[featureName]
                if all([isinstance(x, int) for x in value]):
                    if any([x > 0 for x in value]):
                        value = 1
                    else:
                        value = 0
                    edgeCounts["N"][key] += value
                    for cutoff in [3,10,25]:
                        if edgeNum < cutoff:
                            edgeCounts[cutoff][key] += value
                            
        retval = {}
        for cutoff,cutoffData in edgeCounts.items():
            for key,value in cutoffData.items():
                if retval.get(key) is None:
                    retval[key] = {}
                if isinstance(cutoff, int):
                    retval[key]["".join(["Top", str(cutoff)])] = value
                else:
                    retval[key][cutoff] = value
                    
        if debug:
            print("  Filling edge {0} Counts for {1} Cutoff Values".format(category, len(retval)))
                                
        self.features["Edge_{0}_Counts".format(category)] = retval
        

        

    def fillEdgeInternalCounts(self, debug=False):
        category="Internal"
        if debug:
            print("Filling Edge {0} Counts".format(category))
            
        featureNames = None
            
        from collections import Counter
        edgeCounts = {"N": {}, 3: {}, 10: {}, 25: {}}

        for edgeNum,edgeName in enumerate(self.dn.getEdges()):
            edgeData = dn.getEdgeByName(edgeName, 'feat').get(category)
            if edgeData is None:
                raise ValueError("Could not get {0} category from edge data!".format(category))
            
            if featureNames is None:
                featureNames = list(edgeData.keys())
                for featureName in featureNames:
                    featCats = self.getCategories(featureName)
                    if featCats is not None:
                        if featCats == ['Y', 'N']:
                            key = featureName
                            for cutoff in ["N",3,10,25]:
                                edgeCounts[cutoff][key]  = 0
                        else:
                            for cat in featCats:
                                key = "".join([featureName,cat])
                                for cutoff in ["N",3,10,25]:
                                    edgeCounts[cutoff][key]  = 0
                        
            
            for featureName in featureNames:                
                value = edgeData[featureName]
                featCats = self.getCategories(featureName)
                if featCats is not None:
                    if featCats == ['Y', 'N']:
                        key = featureName
                        edgeCounts["N"][key] += int(value == 'Y')
                        for cutoff in [3,10,25]:
                            if edgeNum < cutoff:
                                edgeCounts[cutoff][key] += int(value == 'Y')
                    elif value in featCats:
                        key = "".join([featureName,value])
                        edgeCounts["N"][key] += 1
                        for cutoff in [3,10,25]:
                            if edgeNum < cutoff:
                                edgeCounts[cutoff][key] += 1
                    else:
                        raise ValueError("Value [{0}] not in [{1}] for feature [{2}]".format(value, featCats, featureName))
                            
                            
        retval = {}
        for cutoff,cutoffData in edgeCounts.items():
            for key,value in cutoffData.items():
                if retval.get(key) is None:
                    retval[key] = {}
                if isinstance(cutoff, int):
                    retval[key]["".join(["Top", str(cutoff)])] = value
                else:
                    retval[key][cutoff] = value
                    
        if debug:
            print("  Filling edge {0} Counts for {1} Cutoff Values".format(category, len(retval)))
                                
        self.features["Edge_{0}_Counts".format(category)] = retval
        
        
        
        
        
    #################################################################################################################
    # Vertex/Edge Properties
    #################################################################################################################
    def fillObjectProperties(self, objectData, debug=False):
        try:
            diffVtx0Vtx1  = float(objectData[0] - objectData[1])
        except:
            diffVtx0Vtx1  = None

        try:
            diffVtx1Vtx2  = float(objectData[1] - objectData[2])
        except:
            diffVtx1Vtx2  = None

        try:
            diffVtx0Vtx12 = float(objectData[0] - objectData[1] - objectData[2])
        except:
            diffVtx0Vtx12 = None

        try:
            qvals = list(objectData.quantile(q=[0.05,0.25,0.5,0.75,0.95]))
        except:
            qvals = [None, None, None, None, None]

        retval = {"Diff_First_Second":  diffVtx0Vtx1,
                  "Diff_Second_Third":  diffVtx1Vtx2,
                  "Diff_Top3":         diffVtx0Vtx12,
                  "Very_Low_Quantile":  qvals[0],
                  "Low_Quantile":      qvals[1],
                  "Mid_Quantile":      qvals[2],
                  "High_Quantile":     qvals[3],
                  "Very_High_Quantile": qvals[4]}
        return retval
        

    def fillVertexProperties(self, debug=False):
        if debug:
            print("Filling Vertex Properties")

        retval = {}
        vertexAttrsDF = self.dn.vInfo.vertexAttrsDF
        dtypes        = vertexAttrsDF.dtypes
        for attribute in vertexAttrsDF.columns:
            if isNumericDtype(dtypes[attribute]):
                vertexData = getColData(vertexAttrsDF, colnames=attribute)            
                retval[attribute] = self.fillObjectProperties(vertexData)

        nodeAttrs = self.dn.getNodeAttrs()
        if not isDataFrame(nodeAttrs):
            if debug:
                print("  There is no NodeAttrs DataFrame")
        else:
            for attribute in nodeAttrs.columns:
                vertexData = getColData(nodeAttrs, colnames=attribute)
                key = "_".join(x.title() for x in attribute.split("_"))
                retval[key] = self.fillObjectProperties(vertexData)

        if debug:
            print("  Filled Vertex Properties for {0} Attributes".format(len(retval)))
            
        self.features["Vertex_Properties"] = retval
        

    def fillEdgeProperties(self, debug=False):
        if debug:
            print("Filling Edge Properties")

        retval      = {}
        edgeAttrsDF = self.dn.eInfo.edgeAttrsDF
        dtypes      = edgeAttrsDF.dtypes
        for attribute in edgeAttrsDF.columns:
            if isNumericDtype(dtypes[attribute]):
                edgeData = getColData(edgeAttrsDF, colnames=attribute)
                retval[attribute] = self.fillObjectProperties(edgeData)

        edgeAttrs = self.dn.getEdgeAttrs()
        if not isDataFrame(edgeAttrs):
            if debug:
                print("  There is no EdgeAttrs DataFrame")
        else:
            for attribute in edgeAttrs.columns:
                vertexData = getColData(edgeAttrs, colnames=attribute)            
                key = "_".join(x.title() for x in attribute.split("_"))
                retval[key] = self.fillObjectProperties(vertexData)

        if debug:
            print("  Filled Edge Properties for {0} Attributes".format(len(retval)))
            
        self.features["Edge_Properties"] = retval

        
        
    #################################################################################################################
    # Top Vertex/Edge Features
    #################################################################################################################
    def fillIndividualObjectFeatures(self, objectNum, objectData, debug=False):
        retval = {}
        retval['Rank'] = objectNum
        for category, categoryData in objectData.items():
            for featureName, featureValue in categoryData.items():
                key = "".join([category,featureName])
                if isinstance(featureValue, list):
                    featureValue = len(featureValue)
                elif isinstance(featureValue, dict):
                    continue

                retval[key] = featureValue
        return retval
    

    def fillIndividualVertexFeatures(self, debug=False):
        if debug:
            print("Filling Individual Vertex Features")
        key = "Vertex_Top5"
        retval = {}
        
        for vertexNum, vertexName in enumerate(dn.getVertices()):
            vertexData = dn.getVertexByName(vertexName, 'feat')
            retval["{0}".format(vertexNum)] = self.fillIndividualObjectFeatures(vertexNum, vertexData, debug=debug)
            break
            
        if debug:
            print("  Filled Individual Features for {0} Vertices".format(len(retval)))
        self.features[key] = retval
        
        
    def fillIndividualEdgeFeatures(self, debug=False):
        if debug:
            print("Filling Individual Edge Features")
        key = "Edge_Top5"
        retval = {}
        
        for edgeNum in range(5):
            edge = dn.getEdge(edgeNum, 'feat')
            retval["{0}".format(edgeNum)] = self.fillIndividualObjectFeatures(edgeNum, edge, debug=debug)
            
        if debug:
            print("  Filled Individual Features for {0} Edges".format(len(retval)))            
        self.features[key] = retval
        
        
    def fillNetworkFeatures(self, debug=False):
        if debug:
            print("Filling Network Features")
        key = "Network"
        retval = {}
        
        netAttrs = self.dn.getNetAttrs()
        for featureName, featureValue in netAttrs.items():
            retval[featureName] = featureValue
            
        if debug:
            print("  Filled {0} Network Features".format(len(retval)))
        self.features[key] = retval


    def fillHomeFeatures(self, debug=False):
        if debug:
            print("Filling Home Vertex Features")
        key = "Home"
        retval = {}
                
        vertexName = str(dn.homeMetrics['GeoID'])
        vertexData = dn.getVertexByName(vertexName, 'feat')
        vertexNum  = dn.getVertexNum(vertexName)
        ratio = dn.homeMetrics['Ratio']
        ratio_significance = self.getHomeRatioCategory(ratio, debug)
        retval["Ratio"]    = ratio_significance
        retval["Days"]     = dn.homeMetrics['Days']
        retval["Days"], _  = self.getIntervalCategory(retval["Days"], debug)
        for featureName, value in self.fillIndividualObjectFeatures(vertexNum, vertexData).items():
            retval[featureName] = value

        if debug:
            print("  Filled {0} Home Vertex Features".format(len(retval)))
            
        self.features[key] = retval
        
                
        
    #################################################################################################################
    # Feature Correlations
    #################################################################################################################
    def fillVertexFeatureCorrelations(self, debug=False):
        if debug:
            print("Filling Vertex Feature Correlations")
        key = "Vertex_Corr"
        retval = {}
        
        vertexAttrs = self.dn.nodeAttrs
        for i,attribute1 in enumerate(vertexAttrs.columns):
            vertexData1 = getColData(vertexAttrs, colnames=attribute1)
            for j,attribute2 in enumerate(vertexAttrs.columns):
                if j <= i:
                    continue
                    
                vertexData2 = getColData(vertexAttrs, colnames=attribute2)               
                try:
                    corr = vertexData1.corr(vertexData2)
                except:
                    corr = None
                retval["_".join([attribute1, attribute2])] = corr

        if debug:
            print("  Filled {0} Vertex Feature Correlations".format(len(retval)))
            
        self.features[key] = retval
        
        
    def fillEdgeFeatureCorrelations(self, debug=False):
        if debug:
            print("Filling Edge Feature Correlations")
        key = "Edge_Corr"
        retval = {}
        
        edgeAttrs = self.dn.edgeAttrs
        for i,attribute1 in enumerate(edgeAttrs.columns):
            edgeData1 = getColData(edgeAttrs, colnames=attribute1)
            for j,attribute2 in enumerate(edgeAttrs.columns):
                if j <= i:
                    continue
                    
                edgeData2 = getColData(edgeAttrs, colnames=attribute2)               
                try:
                    corr = edgeData1.corr(edgeData2)
                except:
                    corr = None
                retval["_".join([attribute1, attribute2])] = corr

        if debug:
            print("  Filled {0} Edge Feature Correlations".format(len(retval)))
            
        self.features[key] = retval
        
        


    

    #######################################################################################################################
    #
    # Create DataFrame
    #
    #######################################################################################################################
    def getRawFeatures(self, debug=False):
        return self.features
    
    
    def getFeatureCategories(self, debug=False):
        return list(self.features.keys())
    
    
    def getFeatures(self, subcategory=None, selfeature=None, debug=False):
        from collections import Counter
        features = {}
        cntr = Counter()
        for category, categorydata in self.features.items():
            if subcategory is not None:
                if category != subcategory:
                    continue
            for feature, featuredata in categorydata.items():
                if isinstance(featuredata, dict):
                    for subfeature, subfeaturedata in featuredata.items():
                        key = "_".join([category,feature,subfeature])
                        key = "".join([s for s in key.split("_")])
                        if selfeature is not None:
                            if selfeature not in key:
                                continue
                        value = fixType(subfeaturedata)
                        features[key] = value
                else:
                    if selfeature is not None:
                        if selfeature not in feature:
                            continue
                    key = "_".join([category,feature])
                    key = "".join([s for s in key.split("_")])
                    value = fixType(featuredata)
                    features[key] = value
        
        if debug:
            print("Created Data Frame with {0} features".format(len(features)))

        if False:
            features['Device'] = self.device
            if self.expectedFeatures is not None:
                if len(features) != self.expectedFeatures:
                    print("\nThere are only {0}/{1} features for {2}!!!\n".format(len(features), self.expectedFeatures, self.device))
                    self.printFeatures()
                    raise ValueError("\nThere are only {0}/{1} features for {2}!!!\n".format(len(features), self.expectedFeatures, self.device))

        return features
    
                        
    def getFeatureDataFrame(self, debug=False):
        from pandas import DataFrame
        features = self.getFeatures(debug=debug)
        df = DataFrame(features, index=[0])
        return df
    
    
    def getHomeFeatureDataFrame(self, debug=False):
        from pandas import DataFrame
        features = self.getFeatures(subcategory="Home", debug=debug)
        df = DataFrame(features, index=[0])
        return df
    
    
    def getCategoryFeatureDataFrame(self, category, debug=False):
        from pandas import DataFrame
        features = self.getFeatures(subcategory=category, debug=debug)
        df = DataFrame(features, index=[0])
        return df
    
    
    def getSubFeatureDataFrame(self, selfeature, debug=False):
        from pandas import DataFrame
        features = self.getFeatures(selfeature=selfeature, debug=debug)
        df = DataFrame(features, index=[0])
        return df
    
    
    def getDwellTimeFeatureDataFrame(self, debug=False):
        from pandas import DataFrame
        features = self.getFeatures(selfeature="DwellTime", debug=debug)
        df = DataFrame(features, index=[0])
        return df

In [204]:
nf = networkFeatures(dn)

## Vertex Counts
nf.fillVertexCensusCounts(debug=True)
nf.fillVertexInternalCounts(debug=True)
nf.fillVertexGeoSpatialCounts(debug=True)
nf.fillVertexProperties(debug=True)

## Edge Counts
nf.fillEdgeInternalCounts(debug=True)
nf.fillEdgeCensusCounts(debug=True)
nf.fillEdgeGeoSpatialCounts(debug=True)
nf.fillEdgeProperties(debug=True)

## Network Counts
nf.fillNetworkFeatures(debug=True)

## Home Counts
nf.fillHomeFeatures(debug=True)

## Indiv Vertex/Edge Values
#nf.fillIndividualVertexFeatures(debug=True)
#nf.fillIndividualEdgeFeatures(debug=True)

## Vertex/Edge Correlations
##nf.fillVertexFeatureCorrelations(debug=True)
#nf.fillEdgeFeatureCorrelations(debug=True)
_,_ = clock("Last Run")

Filling Vertex Census Counts
  Filling Vertex Census Counts
Filling Vertex Internal Counts
  Filling Vertex Internal Counts for 30 Cutoff Values
Filling Vertex GeoSpatial Counts
  Filling Vertex GeoSpatial Counts for 0 Cutoff Values
Filling Vertex Properties
  Filled Vertex Properties for 42 Attributes
Filling Edge Internal Counts
  Filling edge Internal Counts for 29 Cutoff Values
Filling Edge Census Counts
  Filling Edge Census Counts
Filling edge GeoSpatial Counts
  Filling edge GeoSpatial Counts for 104 Cutoff Values
Filling Edge Properties
  Filled Edge Properties for 11 Attributes
Filling Network Features
  Filled 46 Network Features
Filling Home Vertex Features
  Filled 175 Home Vertex Features
Current Time is Tue Dec 04, 2018 15:48:33 for Last Run


In [205]:
nf.getFeatures()

{'VertexCensusCountsCbsaN': 8,
 'VertexCensusCountsCbsaMostCommon': 'Knoxville, TN',
 'VertexCensusCountsCbsaMostCommonFraction': 0.7954545454545454,
 'VertexCensusCountsCbsaTypeN': 3,
 'VertexCensusCountsCbsaTypeMostCommon': 'Metro',
 'VertexCensusCountsCbsaTypeMostCommonFraction': 0.8295454545454546,
 'VertexCensusCountsCbsaPopN': 4,
 'VertexCensusCountsCbsaPopMostCommon': 'Big',
 'VertexCensusCountsCbsaPopMostCommonFraction': 0.7954545454545454,
 'VertexCensusCountsCbsaHousingN': 4,
 'VertexCensusCountsCbsaHousingMostCommon': 'Big',
 'VertexCensusCountsCbsaHousingMostCommonFraction': 0.7954545454545454,
 'VertexCensusCountsCbsaAreaN': 3,
 'VertexCensusCountsCbsaAreaMostCommon': 'Mid',
 'VertexCensusCountsCbsaAreaMostCommonFraction': 0.8068181818181818,
 'VertexCensusCountsCsaN': 5,
 'VertexCensusCountsCsaMostCommon': 'Knoxville-Sevierville-La Follette, TN',
 'VertexCensusCountsCsaMostCommonFraction': 0.8068181818181818,
 'VertexCensusCountsCsaTypeN': 1,
 'VertexCensusCountsCsaTypeMo

In [206]:
nf.getFeatureDataFrame().T

,0
VertexCensusCountsCbsaN,8
VertexCensusCountsCbsaMostCommon,"Knoxville, TN"
VertexCensusCountsCbsaMostCommonFraction,0.795
VertexCensusCountsCbsaTypeN,3
VertexCensusCountsCbsaTypeMostCommon,Metro
VertexCensusCountsCbsaTypeMostCommonFraction,0.83
VertexCensusCountsCbsaPopN,4
VertexCensusCountsCbsaPopMostCommon,Big
VertexCensusCountsCbsaPopMostCommonFraction,0.795
VertexCensusCountsCbsaHousingN,4


In [ ]:
trips['homeMetrics']

# Load Data

# Run the Network

In [ ]:
# Get Sparse Matrix
algosToRun = []
algosToRun.append(linalg.attr_sparse_matrix)
#algosToRun.append(convert_matrix.from_scipy_sparse_matrix)
algosToRun.append(convert_matrix.to_pandas_adjacency)
#runAlgos(algosToRun, g)

In [ ]:
convert_matrix.to_pandas_edgelist(g)

In [ ]:
G = nx.complete_graph(5)
A = nx.nx_agraph.to_agraph(G)
H = nx.nx_agraph.from_agraph(A)

In [ ]:
import pygraphviz as pgv
print(A)

In [ ]:
print(nx.draw(g))

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
import copy
import networkx
import matplotlib.pyplot as plt

# Generate a graph.
# Here I chose an ER graph.
g = nx.erdos_renyi_graph(20, 0.3)

# Get positions.
# Here I use the spectral layout and add a little bit of noise.
pos = nx.layout.spectral_layout(g)
pos = nx.spring_layout(g, pos=pos, iterations=50)

# Create position copies for shadows, and shift shadows
pos_shadow = copy.deepcopy(pos)
shift_amount = 0.006
for idx in pos_shadow:
    pos_shadow[idx][0] += shift_amount
    pos_shadow[idx][1] -= shift_amount

#~~~~~~~~~~~~
# Draw graph
#~~~~~~~~~~~~
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

nx.draw_networkx_nodes(g, pos_shadow, node_color='k', alpha=0.5)
nx.draw_networkx_nodes(g, pos, node_color="#3182bd", linewidths=1)
nx.draw_networkx_edges(g, pos, width=1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

G = nx.Graph(day="Stackoverflow")
df_nodes = pd.read_csv('../input/stack_network_nodes.csv')
df_edges = pd.read_csv('../input/stack_network_links.csv')

for index, row in df_nodes.iterrows():
    G.add_node(row['name'], group=row['group'], nodesize=row['nodesize'])
    
for index, row in df_edges.iterrows():
    G.add_weighted_edges_from([(row['source'], row['target'], row['value'])])
    
color_map = {1:'#f09494', 2:'#eebcbc', 3:'#72bbd0', 4:'#91f0a1', 5:'#629fff', 6:'#bcc2f2',  
             7:'#eebcbc', 8:'#f1f0c0', 9:'#d2ffe7', 10:'#caf3a6', 11:'#ffdf55', 12:'#ef77aa', 
             13:'#d6dcff', 14:'#d2f5f0'} 

plt.figure(figsize=(25,25))
options = {
    'edge_color': '#FFDEA2',
    'width': 1,
    'with_labels': True,
    'font_weight': 'regular',
}
colors = [color_map[G.node[node]['group']] for node in G]
sizes = [G.node[node]['nodesize']*10 for node in G]

"""
Using the spring layout : 
- k controls the distance between the nodes and varies between 0 and 1
- iterations is the number of times simulated annealing is run
default k=0.1 and iterations=50
"""
nx.draw(G, node_color=colors, node_size=sizes, pos=nx.spring_layout(G, k=0.25, iterations=50), **options)
ax = plt.gca()
ax.collections[0].set_edgecolor("#555555") 
plt.show()

In [ ]:
dn.edgeDict[('0', '1')].values()

In [ ]:

    minmaxWeight = [0.0, 2.5]
    print("Number of Edges: {0}".format(nEdges))
    nRange=5
    if nEdges > 100000:
        minmaxWeight[1] = 2
        nRange=6
        weightSize = [power(x,11) for x in linspace(minmaxWeight[0], minmaxWeight[1], nRange)]
    elif nEdges > 50000:
        minmaxWeight[1] = 2
        weightSize = [power(x,9) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 25000:
        weightSize = [power(x,8) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 10000:
        weightSize = [power(x,7) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 2000:
        weightSize = [power(x,6) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 1000:
        weightSize = [power(x,5) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 500:
        weightSize = [power(x,4) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    elif nEdges > 100:
        weightSize = [power(x,3) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    else:
        weightSize = [power(x,2) for x in linspace(minmaxWeight[0], minmaxWeight[1], 5)]
    scale = 2.5/amax(weightSize)
    weightSize = [x*scale for x in weightSize]

In [ ]:
for k,k2,v in g.edges(data=True):
    print(v)
    break
#self.nodeDict = {u: d for (u,d) in self.g.nodes(data=True)}


In [ ]:
from collections import Counter
x = Counter()
x[3] += 1
x

In [ ]:
x.get(4)

In [ ]:
from pandas import Series
tmp = Series([1, 3, 45,6 ,8, 34])

In [ ]:
list(tmp.quantile(q=[0.05,0.95]))

In [ ]:
g.edges(data=True)

In [ ]:
#test

In [ ]:
featCats = ['Y', 'N']

In [ ]:
featCats = 'dkjdflkgjdfgd'

In [ ]:
featCats[:-4]

In [ ]:
featCats[3:]

In [19]:
y = [0, 1]
all([isinstance(x, dict) for x in y])

False